In [29]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".25"

from typing import NamedTuple
import h5py
import numpy as np
import jax
import jax.numpy as jnp
import optax
import haiku as hk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

np.random.seed(8)
print("JAX version {}".format(jax.__version__))
print("Haiku version {}".format(hk.__version__))

JAX version 0.4.8
Haiku version 0.0.9


In [12]:
jax.local_devices()

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]

In [13]:
x = jax.device_put(jnp.array([1,2,3,4,5,6,7,8]))

In [14]:
key = jax.random.PRNGKey(8)

## Read data

In [15]:
# read data
f = h5py.File('/clusterfs/ml4hep/mfong/transfer_learning/delphes_train.h5', 'r')
f2 = h5py.File('/clusterfs/ml4hep/mfong/transfer_learning/delphes_test.h5', 'r')

In [16]:
feature_keys = ['fjet_clus_eta', 'fjet_clus_phi', 'fjet_clus_pt']
for k in f.keys():
    print(k, f[k].shape)

fjet_clus_E (4000068, 200)
fjet_clus_eta (4000068, 200)
fjet_clus_phi (4000068, 200)
fjet_clus_pt (4000068, 200)
fjet_eta (4000068,)
fjet_m (4000068,)
fjet_phi (4000068,)
fjet_pt (4000068,)
labels (4000068,)


In [17]:
x = np.concatenate([f[k] for k in feature_keys], axis=1)
x.shape

(4000068, 600)

In [18]:
scaler = StandardScaler()
scaler.fit(x)

x = scaler.transform(x)

In [19]:
y = f["labels"][:]
y.shape

(4000068,)

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=8)
train_batch_size = 64
eval_batch_size = 128

In [37]:
train_batches = jax.tree_map(lambda x: jnp.array(x, jnp.float32), jax.random.split(jax.random.PRNGKey(0), len(x_train)//train_batch_size))
train_batches = [(train_batches[i], jnp.array(y_train[i:i+train_batch_size], jnp.int32)) for i in range(0, len(train_batches))]
eval_batches = jax.tree_map(lambda x: jnp.array(x, jnp.float32), jax.random.split(jax.random.PRNGKey(1), len(x_test)//eval_batch_size))
eval_batches = [(eval_batches[i], jnp.array(y_test[i:i+eval_batch_size], jnp.int32)) for i in range(0, len(eval_batches))]

In [54]:
x_train.shape

(3200054, 600)

## MLP starter code
https://www.kaggle.com/code/alembcke/titanic-multi-layer-perceptron-using-haiku-jax

In [20]:
class TrainingState(NamedTuple):
    params: hk.Params
    # avg_params: hk.Params
    opt_state: optax.OptState

In [21]:
def net_fn(x: jax.Array) -> jax.Array:
  """Standard MLP network."""
  mlp = hk.Sequential([
      hk.Flatten(),
      hk.Linear(64), jax.nn.relu,
      hk.Linear(8), jax.nn.relu,
      hk.Linear(1),
  ])
  return mlp(x)

In [22]:
network = hk.without_apply_rng(hk.transform(net_fn))
optimiser = optax.adam(1e-2)

In [23]:
def loss(params: hk.Params, features: jnp.ndarray, labels: jnp.ndarray):
    """Loss function, using Sigmoid Binary Cross Entropy loss."""
    logits = network.apply(params, features)
    return optax.sigmoid_binary_cross_entropy(logits, labels).sum(axis=-1).mean()

In [24]:
@jax.jit
def evaluate(params: hk.Params, features: jnp.ndarray, labels: jnp.ndarray):
    """Checks the accuracy of predictions compared to labels."""
    logits = network.apply(params, features)
    predictions = jnp.around(logits, 0)
    return jnp.mean(predictions == labels)

@jax.jit
def update(state: TrainingState, features: jnp.ndarray, labels: jnp.ndarray) -> TrainingState:
    """Learning rule (stochastic gradient descent)."""
    grads = jax.grad(loss)(state.params, features, labels)
    updates, opt_state = optimiser.update(grads, state.opt_state)
    params = optax.apply_updates(state.params, updates)
    # Compute avg_params, the exponential moving average of the "live" params.
    # We use this only for evaluation (cf. https://doi.org/10.1137/0330046).
    # avg_params = optax.incremental_update(params, state.avg_params, step_size=0.001)
    # return TrainingState(params, avg_params, opt_state)
    return TrainingState(params, opt_state)

In [25]:
initial_params = network.init(jax.random.PRNGKey(seed=8), x[0])
initial_opt_state = optimiser.init(initial_params)
state = TrainingState(initial_params, initial_opt_state)
# state = TrainingState(initial_params, initial_params, initial_opt_state)

In [28]:
# print(hk.experimental.tabulate(network)(x[0]))

In [27]:
for step in range(100):
    if step % 10 == 0:
        accuracy = np.array(evaluate(state.params, x[:10000], y[:10000])).item()
        print({"step": step, "accuracy": f"{accuracy:.3f}"})

    # Do SGD on training examples.
    state = update(state, x[:10000], y[:10000])

{'step': 0, 'accuracy': '0.498'}
{'step': 10, 'accuracy': '0.497'}
{'step': 20, 'accuracy': '0.501'}
{'step': 30, 'accuracy': '0.501'}
{'step': 40, 'accuracy': '0.501'}
{'step': 50, 'accuracy': '0.501'}
{'step': 60, 'accuracy': '0.501'}
{'step': 70, 'accuracy': '0.501'}
{'step': 80, 'accuracy': '0.501'}
{'step': 90, 'accuracy': '0.501'}
